In [12]:

# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [13]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [14]:

image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [15]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [17]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [18]:

num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.216333
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.536592
Minibatch accuracy: 50.0%
Validation accuracy: 63.0%
Minibatch loss at step 100: 1.191744
Minibatch accuracy: 56.2%
Validation accuracy: 67.2%
Minibatch loss at step 150: 0.619428
Minibatch accuracy: 81.2%
Validation accuracy: 76.4%
Minibatch loss at step 200: 0.505493
Minibatch accuracy: 87.5%
Validation accuracy: 78.3%
Minibatch loss at step 250: 0.994048
Minibatch accuracy: 68.8%
Validation accuracy: 77.6%
Minibatch loss at step 300: 0.630726
Minibatch accuracy: 75.0%
Validation accuracy: 79.2%
Minibatch loss at step 350: 0.155982
Minibatch accuracy: 93.8%
Validation accuracy: 78.4%
Minibatch loss at step 400: 0.287966
Minibatch accuracy: 100.0%
Validation accuracy: 80.2%
Minibatch loss at step 450: 0.548237
Minibatch accuracy: 87.5%
Validation accuracy: 80.9%
Minibatch loss at step 500: 0.716021
Minibatch accuracy: 87.5%
Validation accuracy: 81.5%


# Problem 1

In [19]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv_1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    bias_1 = tf.nn.relu(conv_1 + layer1_biases)
    max_pool_1=tf.nn.max_pool(bias_1,[1,2,2,1],[1,2,2,1],padding='SAME')
    conv_2 = tf.nn.conv2d(max_pool_1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    bias_2 = tf.nn.relu(conv_2 + layer2_biases)
    max_pool_2=tf.nn.max_pool(bias_2,[1,2,2,1],[1,2,2,1],padding='SAME')
    shape = max_pool_2.get_shape().as_list()
    reshape = tf.reshape(max_pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [21]:


num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.426366
Minibatch accuracy: 12.5%
Validation accuracy: 10.8%
Minibatch loss at step 50: 1.807910
Minibatch accuracy: 25.0%
Validation accuracy: 46.4%
Minibatch loss at step 100: 1.331752
Minibatch accuracy: 56.2%
Validation accuracy: 63.2%
Minibatch loss at step 150: 0.790340
Minibatch accuracy: 68.8%
Validation accuracy: 73.1%
Minibatch loss at step 200: 0.520459
Minibatch accuracy: 87.5%
Validation accuracy: 77.3%
Minibatch loss at step 250: 1.007696
Minibatch accuracy: 62.5%
Validation accuracy: 78.2%
Minibatch loss at step 300: 0.530316
Minibatch accuracy: 75.0%
Validation accuracy: 79.4%
Minibatch loss at step 350: 0.165291
Minibatch accuracy: 93.8%
Validation accuracy: 78.7%
Minibatch loss at step 400: 0.360294
Minibatch accuracy: 87.5%
Validation accuracy: 81.5%
Minibatch loss at step 450: 0.332823
Minibatch accuracy: 87.5%
Validation accuracy: 82.3%
Minibatch loss at step 500: 0.576964
Minibatch accuracy: 87.5%
Validation accuracy: 82.7%
M

# Problem 2

In [76]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

dropout = 0.5
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  size = image_size // 4 * image_size // 4 * depth
  layer3_weights = tf.Variable(tf.truncated_normal(
      [size, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  # Model.
  def model(data,prob):
    conv_1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    bias_1 = tf.nn.relu(conv_1 + layer1_biases)
    max_pool_1=tf.nn.max_pool(bias_1,[1,2,2,1],[1,2,2,1],padding='SAME')
    conv_2 = tf.nn.conv2d(max_pool_1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    bias_2 = tf.nn.relu(conv_2 + layer2_biases)
    max_pool_2=tf.nn.max_pool(bias_2,[1,2,2,1],[1,2,2,1],padding='SAME')
    shape = max_pool_2.get_shape().as_list()
    reshape = tf.reshape(max_pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    drop=tf.nn.dropout(hidden,prob)
    hidden2 = tf.nn.relu(tf.matmul(hidden, layer4_weights) + layer4_biases)
    drop2=tf.nn.dropout(hidden2,prob)
    return tf.matmul(drop2, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset,dropout)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  # Optimizer.
  
  learning_rate = tf.train.exponential_decay(0.05, global_step, 2400, 0.85, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset,1.0))
  test_prediction = tf.nn.softmax(model(tf_test_dataset,1.0))

In [77]:
num_steps = 24001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.610913
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.359652
Minibatch accuracy: 18.8%
Validation accuracy: 17.4%
Minibatch loss at step 100: 1.861758
Minibatch accuracy: 56.2%
Validation accuracy: 40.5%
Minibatch loss at step 150: 1.789557
Minibatch accuracy: 43.8%
Validation accuracy: 55.0%
Minibatch loss at step 200: 0.851236
Minibatch accuracy: 68.8%
Validation accuracy: 71.0%
Minibatch loss at step 250: 1.511195
Minibatch accuracy: 62.5%
Validation accuracy: 75.5%
Minibatch loss at step 300: 0.965806
Minibatch accuracy: 68.8%
Validation accuracy: 76.6%
Minibatch loss at step 350: 0.336611
Minibatch accuracy: 93.8%
Validation accuracy: 78.2%
Minibatch loss at step 400: 0.618172
Minibatch accuracy: 81.2%
Validation accuracy: 79.9%
Minibatch loss at step 450: 0.559413
Minibatch accuracy: 81.2%
Validation accuracy: 80.3%
Minibatch loss at step 500: 0.782047
Minibatch accuracy: 81.2%
Validation accuracy: 79.8%
Mi